In [1]:
import pandas as pd
import os
from constants import TASK_MODEL_PATH, TOKENIZER_SUFFIXES, MOLNET_DIRECTORY, PROJECT_PATH, RETROSYNTHESIS_DIRECTORY, DESCRIPTORS

from glob import glob
from pathlib import Path
pd.set_option('display.max_rows', 500)

classification_scores = []
regression_scores = []
TOKENIZER_SUFFIXES+=["smiles_isomers_atom","smiles_isomers_sentencepiece","selfies_isomers_atom","selfies_isomers_sentencepiece"]
for tokenizer_suffix in TOKENIZER_SUFFIXES:
    for task in MOLNET_DIRECTORY.keys():
        task_tokenizer_path = TASK_MODEL_PATH/task/tokenizer_suffix
        if task.startswith("bace") and not ("isomer" in tokenizer_suffix) :
            task_tokenizer_path = Path("/data/jgut/SoS_models/task")/task/tokenizer_suffix
        print(task_tokenizer_path)
        if task_tokenizer_path.exists():
            for hyperparameter_path in glob(str(task_tokenizer_path) + "/*", recursive=True):
                scores_path = hyperparameter_path+"/scores.csv"
                print(scores_path)
                if not Path(scores_path).is_file():
                    continue
                new_score_df = pd.read_csv(scores_path)
                if list(new_score_df.task_type)[0] == "classification":
                    classification_scores.append(new_score_df)
                else:
                    regression_scores.append(new_score_df)

regression_scores = pd.concat(regression_scores, axis = 0, sort = False)
classification_scores = pd.concat(classification_scores, axis = 0, sort = False)

/home/jgut/miniconda3/envs/SoS3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. cannot import name 'DMPNN' from 'deepchem.models.torch_models' (/home/jgut/miniconda3/envs/SoS3/lib/python3.10/site-packages/deepchem/models/torch_models/__init__.py)
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'pytorch_lightning'
Skipped loading some Jax models, missing a dependency. No module named 'jax'


/data/jgut/SoS_models/task/bace_classification/selfies_sentencepiece
/data/jgut/SoS_models/task/bace_classification/selfies_sentencepiece/5e-06_0.1_based_norm/scores.csv
/data/jgut/SoS_models/task/bace_classification/selfies_sentencepiece/5e-06_0.3_based_norm/scores.csv
/data/jgut/SoS_models/task/bace_classification/selfies_sentencepiece/5e-05_0.1_based_norm/scores.csv
/data/jgut/SoS_models/task/bace_classification/selfies_sentencepiece/5e-05_0.3_based_norm/scores.csv
/data/jgut/SoS_models/task/bace_classification/selfies_sentencepiece/1e-05_0.1_based_norm/scores.csv
/data/jgut/SoS_models/task/bace_classification/selfies_sentencepiece/1e-05_0.3_based_norm/scores.csv
/data/jgut/SoS_models/task/bace_classification/selfies_sentencepiece/train.label/scores.csv
/data/jgut/SoS_models/task/bace_classification/selfies_sentencepiece/5e-05_0.2_based_norm/scores.csv
/data/jgut/SoS_models/task/bace_classification/selfies_sentencepiece/input0/scores.csv
/data/jgut/SoS_models/task/bace_classificatio

/data/jgut/SoS_models/lipo/smiles_atom/5e-05_0.1_based_norm/scores.csv
/data/jgut/SoS_models/lipo/smiles_atom/5e-05_0.3_based_norm/scores.csv
/data/jgut/SoS_models/lipo/smiles_atom/1e-05_0.1_based_norm/scores.csv
/data/jgut/SoS_models/lipo/smiles_atom/1e-05_0.3_based_norm/scores.csv
/data/jgut/SoS_models/lipo/smiles_atom/5e-05_0.2_based_norm/scores.csv
/data/jgut/SoS_models/lipo/smiles_atom/1e-05_0.2_based_norm/scores.csv
/data/jgut/SoS_models/lipo/smiles_atom/5e-06_0.2_based_norm/scores.csv
/data/jgut/SoS_models/tox21/smiles_atom
/data/jgut/SoS_models/tox21/smiles_atom/5e-06_0.1_based_norm/scores.csv
/data/jgut/SoS_models/tox21/smiles_atom/5e-06_0.3_based_norm/scores.csv
/data/jgut/SoS_models/tox21/smiles_atom/5e-05_0.1_based_norm/scores.csv
/data/jgut/SoS_models/tox21/smiles_atom/5e-05_0.3_based_norm/scores.csv
/data/jgut/SoS_models/tox21/smiles_atom/1e-05_0.1_based_norm/scores.csv
/data/jgut/SoS_models/tox21/smiles_atom/1e-05_0.3_based_norm/scores.csv
/data/jgut/SoS_models/tox21/smi

/data/jgut/SoS_models/clintox/selfies_isomers_atom/5e-06_0.1_based_norm/scores.csv
/data/jgut/SoS_models/clintox/selfies_isomers_atom/5e-06_0.3_based_norm/scores.csv
/data/jgut/SoS_models/clintox/selfies_isomers_atom/5e-05_0.1_based_norm/scores.csv
/data/jgut/SoS_models/clintox/selfies_isomers_atom/5e-05_0.3_based_norm/scores.csv
/data/jgut/SoS_models/clintox/selfies_isomers_atom/1e-05_0.1_based_norm/scores.csv
/data/jgut/SoS_models/clintox/selfies_isomers_atom/1e-05_0.3_based_norm/scores.csv
/data/jgut/SoS_models/clintox/selfies_isomers_atom/5e-05_0.2_based_norm/scores.csv
/data/jgut/SoS_models/clintox/selfies_isomers_atom/1e-05_0.2_based_norm/scores.csv
/data/jgut/SoS_models/clintox/selfies_isomers_atom/5e-06_0.2_based_norm/scores.csv
/data/jgut/SoS_models/delaney/selfies_isomers_atom
/data/jgut/SoS_models/delaney/selfies_isomers_atom/5e-06_0.1_based_norm/scores.csv
/data/jgut/SoS_models/delaney/selfies_isomers_atom/5e-06_0.3_based_norm/scores.csv
/data/jgut/SoS_models/delaney/selfie

In [2]:
columns = list(regression_scores.columns[-3:])
columns.extend(regression_scores.columns[:-3])

regression_scores.sort_values(["task", "tokenizer"])[columns].sort_values(["mean_absolute_error"]).drop(["model_size", "Unnamed: 0", "task_type"], axis="columns").groupby(["task", "tokenizer"]).first()

mean_absolute_error  max_error  \
task            tokenizer                                                       
bace_regression selfies_atom                              0.665644   2.313257   
                selfies_isomers_atom                      0.645083   2.893325   
                selfies_isomers_sentencepiece             0.611286   2.172028   
                selfies_sentencepiece                     1.112284   3.583172   
                smiles_atom                               1.072521   3.018604   
                smiles_isomers_atom                       0.576228   2.750119   
                smiles_isomers_sentencepiece              0.657473   2.928247   
                smiles_sentencepiece                      1.078452   3.000588   
clearance       selfies_atom                              1.019760   2.862548   
                selfies_isomers_atom                      0.872791   3.239782   
                selfies_isomers_sentencepiece             1.054104   3.461823   
                selfies_sentencepiece                     1.076179   3.079184   
                smiles_atom                               0.958213   3.217534   
                smiles_isomers_atom                       0.872958   3.102813   
                smiles_isomers_sentencepiece              1.101552   2.415144   
                smiles_sentencepiece                      1.009197   2.746415   
delaney         selfies_atom                              0.394425   1.533072   
                selfies_isomers_atom                      0.407532   1.961080   
                selfies_isomers_sentencepiece             0.485352   1.756895   
                selfies_sentencepiece                     0.506865   1.417774   
                smiles_atom                               0.360438   1.585986   
                smiles_isomers_atom                       0.359700   1.628031   
                smiles_isomers_sentencepiece              0.502597   2.089880   
                smiles_sentencepiece                      0.580551   2.154503   
lipo            selfies_atom                              0.689535   2.776778   
                selfies_isomers_atom                      0.549668   2.478325   
                selfies_isomers_sentencepiece             0.636758   2.612354   
                selfies_sentencepiece                     0.640858   3.102017   
                smiles_atom                               0.623058   2.433880   
                smiles_isomers_atom                       0.517186   2.647385   
                smiles_isomers_sentencepiece              0.629662   2.882326   
                smiles_sentencepiece                      0.649462   2.050359   

                                               mean_squared_error  \
task            tokenizer                                           
bace_regression selfies_atom                             0.688964   
                selfies_isomers_atom                     0.657136   
                selfies_isomers_sentencepiece            0.566057   
                selfies_sentencepiece                    1.714706   
                smiles_atom                              1.644988   
                smiles_isomers_atom                      0.537215   
                smiles_isomers_sentencepiece             0.667397   
                smiles_sentencepiece                     1.587879   
clearance       selfies_atom                             1.578902   
                selfies_isomers_atom                     1.260572   
                selfies_isomers_sentencepiece            1.747017   
                selfies_sentencepiece                    1.593290   
                smiles_atom                              1.468817   
                smiles_isomers_atom                      1.246937   
                smiles_isomers_sentencepiece             1.533561   
                smiles_sentencepiece                     1.391661   
delaney         selfies_atom                      

In [3]:
columns = list(classification_scores.columns[-3:])
columns.extend(classification_scores.columns[:-3])

classification_scores.sort_values(["task", "tokenizer"])[columns].sort_values(["ROC_AUC"], ascending=False).drop(["model_size", "Unnamed: 0", "task_type"], axis="columns").groupby(["task", "tokenizer"]).first()

ROC_AUC  \
task                tokenizer                                 
bace_classification selfies_atom                   0.788043   
                    selfies_isomers_atom           0.815217   
                    selfies_isomers_sentencepiece  0.748913   
                    selfies_sentencepiece          0.634058   
                    smiles_atom                    0.749638   
                    smiles_isomers_atom            0.816304   
                    smiles_isomers_sentencepiece   0.767935   
                    smiles_sentencepiece           0.666123   
bbbp                selfies_atom                   0.708739   
                    selfies_isomers_atom           0.731092   
                    selfies_isomers_sentencepiece  0.699875   
                    selfies_sentencepiece          0.691589   
                    smiles_atom                    0.714616   
                    smiles_isomers_atom            0.691300   
                    smiles_isomers_sentencepiece   0.697081   
                    smiles_sentencepiece           0.733308   
clintox             selfies_atom                   0.734058   
                    selfies_isomers_atom           0.681159   
                    selfies_isomers_sentencepiece  0.679710   
                    selfies_sentencepiece          0.660870   
                    smiles_atom                    0.755797   
                    smiles_isomers_atom            0.800000   
                    smiles_isomers_sentencepiece   0.677536   
                    smiles_sentencepiece           0.588406   
hiv                 selfies_atom                   0.742450   
                    selfies_isomers_atom           0.775032   
                    selfies_isomers_sentencepiece  0.746028   
                    selfies_sentencepiece          0.744519   
                    smiles_atom                    0.703903   
                    smiles_isomers_atom            0.760586   
                    smiles_isomers_sentencepiece   0.753502   
                    smiles_sentencepiece           0.763884   
tox21               selfies_atom                   0.709894   
                    selfies_isomers_atom           0.695673   
                    selfies_isomers_sentencepiece  0.660249   
                    selfies_sentencepiece          0.649169   
                    smiles_atom                    0.621898   
                    smiles_isomers_atom            0.674938   
                    smiles_isomers_sentencepiece   0.656718   
                    smiles_sentencepiece           0.552454   

                                                   average_precision  \
task                tokenizer                                          
bace_classification selfies_atom                            0.823585   
                    selfies_isomers_atom                    0.872390   
                    selfies_isomers_sentencepiece           0.774967   
                    selfies_sentencepiece                   0.699327   
                    smiles_atom                             0.792704   
                    smiles_isomers_atom                     0.857697   
                    smiles_isomers_sentencepiece            0.793933   
                    smiles_sentencepiece                    0.744993   
bbbp                selfies_atom                            0.636280   
                    selfies_isomers_atom                    0.654241   
                    selfies_isomers_sentencepiece           0.629645   
                    selfies_sentencepiece                   0.668870   
                    smiles_atom                             0.655061   
                    smiles_isomers_atom                     0.628493   
                    smiles_isomers_sentencepiece            0.649734   
                    smiles_sentencepiece                    0.687772   
clintox             selfies_atom                            0.209268   
                    selfies_isomers_a

In [21]:
import pandas as pd
from constants import REACTION_PREDICTION_DIRECTORY, TOKENIZER_SUFFIXES, PROJECT_PATH
pd.options.display.float_format = '{:,.3f}'.format
dfs = []
for task in REACTION_PREDICTION_DIRECTORY:
    for tokenizer in TOKENIZER_SUFFIXES:
        dfs.append(pd.read_csv(PROJECT_PATH/"reaction_prediction_beam"/task/tokenizer/"output.csv"))
df = pd.concat(dfs)
df["perc_0"]=0
for i in range(1,11):
    df["perc_"+str(i)] =df["top_"+str(i)]/df["all_samples"]+df["perc_"+str(i-1)]
df.sort_values(["task", "model"])[[i for i in df.columns if ("len" in i and "mean" in i) or ("nw" in i and "mean" in i) or ("lev" in i and "mean" in i) or ("dl" in i and "mean" in i) or i in ["model", "task"]]].to_markdown()

'|    | model                 | task      |   source_len_mean |   target_len_mean |   max_len_mean |   len_diff_mean |   nw_mean |   nw_norm_mean |   lev_mean |   lev_norm_mean |   dl_mean |   dl_norm_mean |\n|---:|:----------------------|:----------|------------------:|------------------:|---------------:|----------------:|----------:|---------------:|-----------:|----------------:|----------:|---------------:|\n|  0 | selfies_atom          | jin       |           64.1012 |          39.9841  |        64.163  |        24.2407  |  -2.55768 |     0.00698339 |    35.7375 |        0.537783 |   34.2662 |       0.512362 |\n|  0 | selfies_sentencepiece | jin       |           35.3185 |          15.885   |        64.163  |        24.2407  |  -2.55768 |     0.00698339 |    35.7375 |        0.537783 |   34.2662 |       0.512362 |\n|  0 | smiles_atom           | jin       |           66.252  |          40.6978  |        66.2585 |        25.5673  |  -2.73844 |     0.00813676 |    36.9929 |        

In [17]:
df.columns

Index(['Unnamed: 0', 'model', 'task', 'all_samples', 'top_1', 'top_2', 'top_3',
       'top_4', 'top_5', 'top_6', 'top_7', 'top_8', 'top_9', 'top_10',
       'source_len_mean', 'source_len_median', 'source_len_std',
       'target_len_mean', 'target_len_median', 'target_len_std',
       'max_len_mean', 'max_len_median', 'max_len_std', 'len_diff_mean',
       'len_diff_median', 'len_diff_std', 'nw_mean', 'nw_median', 'nw_std',
       'nw_norm_mean', 'nw_norm_median', 'nw_norm_std', 'lev_mean',
       'lev_median', 'lev_std', 'lev_norm_mean', 'lev_norm_median',
       'lev_norm_std', 'dl_mean', 'dl_median', 'dl_std', 'dl_norm_mean',
       'dl_norm_median', 'dl_norm_std', 'rouge1_mean', 'rouge1_median',
       'rouge1_std', 'rouge2_mean', 'rouge2_median', 'rouge2_std',
       'rouge3_mean', 'rouge3_median', 'rouge3_std', 'rougeL_mean',
       'rougeL_median', 'rougeL_std', 'BLEU_mean', 'BLEU_median', 'BLEU_std',
       'BLEU1_mean', 'BLEU1_median', 'BLEU1_std', 'BLEU2_mean', 'BLEU2_media

In [ ]:
descriptors = [ 'NumAliphaticCarbocycles',
 'NumAliphaticHeterocycles',
 'NumAliphaticRings',
 'NumAromaticCarbocycles',
 'NumAromaticHeterocycles',
 'NumAromaticRings',
 'NumHAcceptors',
 'NumHDonors',
 'NumHeteroatoms',
 'NumRotatableBonds',
 'NumSaturatedCarbocycles',
 'NumSaturatedHeterocycles',
 'NumSaturatedRings',
 'RingCount',]
indexes = [str(DESCRIPTORS.index(descriptor)) for descriptor in descriptors]
test = pd.read_csv("processed/10m_deduplicated.csv", skiprows=0, usecols=indexes)
for index, descriptor in enumerate(descriptors):
    amount = sum(test[indexes[index]].gt(0))
    print(f"The stats for descriptor {descriptor}")
    print(f"Amount of mols with trait: {amount}, mols without trait: {len(test)-amount}")
    print(f"This is {amount/len(test):.3f}, mols without trait: {1-amount/len(test):.3f}")